In [1]:
import pandas as pd

In [3]:
labels = ['fake', 'junksci', 'bias', 'unknown', 'conspiracy',
       'political', 'unreliable', 'hate', 'clickbait', 'reliable',
       'rumor', 'satire']

In [36]:

df = pd.read_csv('E:/ML/fake_news_data/news.csv/news_cleaned_2018_02_13.csv', chunksize=1000, index_col=0, lineterminator='\n')

In [30]:
rows_number = 0
for chunk in df:
    rows_number += len(chunk)

print(rows_number)


8528956


In [37]:
import concurrent.futures
import os

# Create a thread pool
# Submit the generate_dataset function for each chunk of article links

def start_thread(chunk):
    executor = ThreadPoolExecutor(max_workers=threads)
    futures = executor.submit(lambda x : len(x), chunk)
    executor.shutdown(wait=True)
    return(futures)

for chunk in df:
    futures = start_thread(chunk)